# Task 1
This notebook contains the solution for Task 1, splitted according to the logical parts used during writing

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [179]:
# Read different tables
meters = pd.read_excel("gorilla_test_data.xlsx", sheet_name="meter_list")
forecasts = pd.read_excel("gorilla_test_data.xlsx", sheet_name="forecast_table")
rates = pd.read_excel("gorilla_test_data.xlsx", sheet_name="rate_table")

# Merge forecasts with the meters
forecasts = forecasts.merge(right=meters, on="meter_id")
forecasts.head()

,meter_id,date,kwh,aq_kwh,exit_zone
0,14676236,2020-06-01,22.070768,28978,EA1
1,14676236,2020-06-02,19.170720,28978,EA1
2,14676236,2020-06-03,23.555111,28978,EA1
3,14676236,2020-06-04,18.220712,28978,EA1
4,14676236,2020-06-05,14.196134,28978,EA1


In [180]:
# Merge rates with meters and get correct rates
rates = rates.merge(right=meters, on="exit_zone")
rates = rates[((rates.aq_kwh >= rates.aq_min_kwh) & (rates.aq_kwh < rates.aq_max_kwh)) | ((rates.aq_max_kwh.isnull()) & (rates.aq_kwh >= rates.aq_min_kwh))]
rates = rates.drop(["aq_min_kwh", "aq_max_kwh", "exit_zone"], axis=1)
rates.head()

,date,rate_p_per_kwh,meter_id,aq_kwh
0,2020-04-01,0.2652,14676236,28978
3,2020-10-01,0.2970,14676236,28978
6,2021-04-01,0.3327,14676236,28978
9,2021-10-01,0.3726,14676236,28978
12,2022-04-01,0.4173,14676236,28978


In [181]:
# Get all daily rates for all forecasts

# Get list of all dates
dates = sorted(rates.date.unique())

# Get index where date should be inserted to maintain order of dates
date_indices = np.searchsorted(dates, forecasts.date, side="right") - 1

# Add start date of period
forecasts["rate_period"] = [dates[i] for i in date_indices]
forecasts.head()


,meter_id,date,kwh,aq_kwh,exit_zone,rate_period
0,14676236,2020-06-01,22.070768,28978,EA1,2020-04-01
1,14676236,2020-06-02,19.170720,28978,EA1,2020-04-01
2,14676236,2020-06-03,23.555111,28978,EA1,2020-04-01
3,14676236,2020-06-04,18.220712,28978,EA1,2020-04-01
4,14676236,2020-06-05,14.196134,28978,EA1,2020-04-01


In [182]:
# Combine forcasts with actual rates
forecasts = forecasts.merge(right=rates, right_on=["date", "meter_id"], left_on=["rate_period", "meter_id"])

In [183]:
# Calculate daily rate (in p)
forecasts["daily_rate"] = forecasts.kwh * forecasts.rate_p_per_kwh
forecasts.head()

,meter_id,date_x,kwh,aq_kwh_x,exit_zone,rate_period,date_y,rate_p_per_kwh,aq_kwh_y,daily_rate
0,14676236,2020-06-01,22.070768,28978,EA1,2020-04-01,2020-04-01,0.2652,28978,5.853168
1,14676236,2020-06-02,19.170720,28978,EA1,2020-04-01,2020-04-01,0.2652,28978,5.084075
2,14676236,2020-06-03,23.555111,28978,EA1,2020-04-01,2020-04-01,0.2652,28978,6.246815
3,14676236,2020-06-04,18.220712,28978,EA1,2020-04-01,2020-04-01,0.2652,28978,4.832133
4,14676236,2020-06-05,14.196134,28978,EA1,2020-04-01,2020-04-01,0.2652,28978,3.764815


In [194]:
# Calculate total per meter_id
totals_per_meter = forecasts.groupby(forecasts.meter_id).sum()[["kwh", "daily_rate"]]
totals_per_meter.daily_rate = totals_per_meter.daily_rate / 100
totals_per_meter.columns = ["Total Estimated consumption (kWh)", "Total Cost (£)"]
totals_per_meter.round(2)


,Total Estimated consumption (kWh),Total Cost (£)
meter_id,,
14676236,28978.0,100.15
34509937,78324.0,275.49
50264822,265667.0,731.24
88357331,484399.0,1433.16
